In [ ]:
# Code taken and adapted from : https://github.com/jasonwei20/eda_nlp/tree/master/experiments

In [1]:
from methods import *

In [10]:
## config file
datasets = ['sst']
dataset_folders = ['increment_datasets_f2/' + dataset for dataset in datasets] 
num_classes_list = [2]
input_size_list = [50]
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300
increments = [0.7, 0.8, 0.9, 1]

In [8]:
#pre-existing file locations
dataset_folder = "sst"

train_orig = dataset_folder + '/train_orig.txt'

#file to be created
train_aug_st = dataset_folder + '/train_aug_st.txt'

#standard augmentation
gen_standard_aug(train_orig, train_aug_st)

#generate the vocab dictionary
word2vec_pickle = dataset_folder + '/word2vec.p' # don't want to load the huge pickle every time, so just save the words that are actually used into a smaller dictionary
gen_vocab_dicts(dataset_folder, word2vec_pickle, huge_word2vec)

finished eda for sst/train_orig.txt to sst/train_aug_st.txt
['sst/train_aug_st.txt', 'sst/train_orig.txt']
73655 unique words found
31950 matches between unique words and word2vec dictionary
dictionaries outputted to sst/word2vec.p


In [5]:
###############################
#### run model and get acc ####
###############################

def run_model(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_model(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=100000, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=0)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y = None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [12]:
#get the accuracy at each increment
## TODO: there is some problemo , fix first the directories problem, coming from first cells, then verify we are properly using the correct dataset and identify where the problem is coming from

orig_accs = {dataset:{} for dataset in datasets}
aug_accs = {dataset:{} for dataset in datasets}

writer = open('outputs_f2/' + get_now_str() + '.csv', 'w')

#for each dataset
for i, dataset_folder in enumerate(dataset_folders):

    dataset = datasets[i]
    num_classes = num_classes_list[i]
    input_size = input_size_list[i]
    train_orig = 'sst/train_orig.txt' #dataset_folder + '/train_orig.txt'
    train_aug_st = 'sst/train_aug_st.txt' #dataset_folder + '/train_aug_st.txt'
    test_path = 'sst/test.txt'#dataset_folder + '/test.txt'
    word2vec_pickle = 'sst/word2vec.p' ## dataset_folder + '/word2vec.p' TODO: check this
    word2vec = load_pickle(word2vec_pickle)

    for increment in increments:

        #calculate augmented accuracy
        aug_acc = run_model(train_aug_st, test_path, num_classes, increment)
        aug_accs[dataset][increment] = aug_acc

        #calculate original accuracy
        orig_acc = run_model(train_orig, test_path, num_classes, increment)
        orig_accs[dataset][increment] = orig_acc

        print(dataset, increment, orig_acc, aug_acc)
        writer.write(dataset + ',' + str(increment) + ',' + str(orig_acc) + ',' + str(aug_acc) + '\n')

        gc.collect()

print(orig_accs, aug_accs)

IndexError: index 10831 is out of bounds for axis 0 with size 2